In [84]:
import esper
import itertools

*director* manages the *scene* state, the *world* manages the *processor* state

class have instances of:

`Director -> Scene -> World -> Processor`

class contain references to:

`Processor -> World -> Scene -> Director`

can change game state from processor:

`self.world.change_processors('life')`

can change scene state from processor:

`self.world.scene.director.change_scene('scene')`

In [85]:
class Director:
    def __init__(self):
        self.scenes = {
            'scene': Scene(),
            'menu': None
        }
        self.current_scene = self.scenes['scene']
        Scene.director = self
    
    def change_scene(self, state):
        self.current_scene = self.scenes[state]
    
    def update(self):
        self.current_scene.update()


In [86]:
class Scene:
    director = None

    def __init__(self):
        self.world = World()
        World.scene = self
    
    def update(self):
        self.world.process()


In [87]:
# jerry rig a 'state machine' into the esper world.
# self._processors is the current processors used in the game state


class World(esper.World):
    scene = None

    def __init__(self):
        super().__init__()
        self._define_processors()
        self._define_world_reference()
        self._processors = self.processors['sleep']
    
    def _define_processors(self):
        sleep = Sleep()
        wake_up = WakeUp()
        hello = Hello()
        shake_hand = ShakeHand()
        goodbye = GoodBye()

        self.processors = {
            'sleep': [
                sleep,
                sleep,
                sleep,
            ],
            'wake_up': [
                wake_up,
            ],
            'life': [
                hello,
                shake_hand,
                goodbye]
        }
    
    def _define_world_reference(self):
        nested_proc = [p for p in self.processors.values()]
        unique_proc = set(itertools.chain.from_iterable(nested_proc))
        for processor_instance in unique_proc:
            processor_instance.world = self
        
    def change_processors(self, state):
        self._processors = self.processors[state]


In [88]:
class Sleep(esper.Processor):
    
    def __init__(self):
        super().__init__()
        
    def process(self, *args):
        print('sleep')
        self.world.change_processors('wake_up')


class WakeUp(esper.Processor):
    
    def __init__(self):
        super().__init__()
        
    def process(self, *args):
        print('wake_up')
        self.world.change_processors('life')


class Hello(esper.Processor):
    
    def __init__(self):
        super().__init__()
        
    def process(self, *args):
        print('hello')
        

class ShakeHand(esper.Processor):
    
    def __init__(self):
        super().__init__()
        
    def process(self, *args):
        print('shake_hand')

        
class GoodBye(esper.Processor):
    
    def __init__(self):
        super().__init__()
        
    def process(self, *args):
        print('goodbye')
        self.world.change_processors('sleep')


In [89]:
s = SceneManager()
s.update()
s.update()
s.update()
s.update()


sleep
sleep
sleep
wake_up
hello
shake_hand
goodbye
sleep
sleep
sleep
